In [1]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

'\nCopyright (c) Microsoft Corporation.\n'

In [1]:
import os

import pandas as pd
import tiktoken

from graphrag.query.indexer_adapters import read_indexer_entities, read_indexer_reports
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch

## Global Search example

Global search method generates answers by searching over all AI-generated community reports in a map-reduce fashion. This is a resource-intensive method, but often gives good responses for questions that require an understanding of the dataset as a whole (e.g. What are the most significant values of the herbs mentioned in this notebook?).

### LLM setup

In [2]:
api_key = os.environ["GRAPHRAG_API_KEY"]
llm_model = os.environ["GRAPHRAG_LLM_MODEL"]
embedding_model = os.environ["GRAPHRAG_EMBEDDING_MODEL"]
api_base = os.environ["GRAPHRAG_API_BASE"]
api_version = os.environ["GRAPHRAG_API_VERSION"]

llm = ChatOpenAI(
    api_key=api_key,
    api_base=api_base,
    api_version=api_version,
    model=llm_model,
    api_type=OpenaiApiType.AzureOpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

### Load community reports as context for global search

- Load all community reports in the `create_final_community_reports` table from the ire-indexing engine, to be used as context data for global search.
- Load entities from the `create_final_nodes` and `create_final_entities` tables from the ire-indexing engine, to be used for calculating community weights for context ranking. Note that this is optional (if no entities are provided, we will not calculate community weights and only use the `rank` attribute in the community reports table for context ranking)

In [3]:
# parquet files generated from indexing pipeline
INPUT_DIR = "../graphfleet/output/graphindex/artifacts"
COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"

# community level in the Leiden community hierarchy from which we will load the community reports
# higher value means we use reports from more fine-grained communities (at the cost of higher computation cost)
COMMUNITY_LEVEL = 2

In [4]:
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)
print(f"Total report count: {len(report_df)}")
print(
    f"Report count after filtering by community level {COMMUNITY_LEVEL}: {len(reports)}"
)
report_df.head()

Total report count: 102
Report count after filtering by community level 2: 83


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,100,# Natural Language Processing and Information ...,2,8.5,Natural Language Processing and Information Re...,The rating is high due to the significant impa...,The community of 'Natural Language Processing ...,[{'explanation': 'Community summaries play a p...,"{\n ""title"": ""Natural Language Processing a...",8c20fd3b-1450-4841-a64a-18429d05d5d3
1,101,# Natural Language Processing and Information ...,2,9.0,Natural Language Processing and Information Re...,The text is highly significant and impactful i...,The community of Natural Language Processing (...,[{'explanation': 'Recent advancements in trans...,"{\n ""title"": ""Natural Language Processing a...",cea459eb-b1cf-46cd-aabb-0ce4f3c184b7
2,71,# Natural Language Processing and Information ...,2,9.0,Natural Language Processing and Information Re...,The text is highly significant and impactful i...,The community of 'Natural Language Processing ...,[{'explanation': 'The 'NEWS ARTICLES' dataset ...,"{\n ""title"": ""Natural Language Processing a...",1458f6ef-6da1-486e-94d1-3070e4e280ff
3,72,# Natural Language Processing and Information ...,2,9.0,Natural Language Processing and Information Re...,The text is highly significant and impactful i...,The community of 'Natural Language Processing ...,[{'explanation': 'The 'PODCAST TRANSCRIPTS' da...,"{\n ""title"": ""Natural Language Processing a...",bcb2afbf-6b77-4e7c-ad5f-845d873cd10d
4,73,# Graph RAG and Community Summarization in NLP...,2,9.0,Graph RAG and Community Summarization in NLP a...,The rating is high due to the significant impa...,The community revolves around the Graph RAG sy...,[{'explanation': 'Graph RAG is a pivotal syste...,"{\n ""title"": ""Graph RAG and Community Summa...",6a14f684-f45d-4563-88f7-18a2cac9b5cb


#### Build global context based on community reports

In [5]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

#### Perform global search

In [6]:
context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 2000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.0,
}

In [7]:
search_engine = GlobalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=False,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [9]:
result = await search_engine.asearch(
    "What is the purpose of GraphRAG?"
)

print(result.response)

### Purpose of GraphRAG

GraphRAG (Graph-based Retrieval-Augmented Generation) is designed to enhance the capabilities of retrieval-augmented generation tasks by integrating retrieval mechanisms with generation models. This approach allows for more comprehensive and contextually relevant summaries by retrieving pertinent information from large datasets before generating the final summary [Data: Reports (57, 22, 70, 68, 71)].

### Key Features and Benefits

1. **Graph-Based Text Indexing**:
   GraphRAG leverages graph-based text indexing to partition data, facilitating global summarization. This method enhances the efficiency and effectiveness of text data processing and summarization by utilizing the natural modularity of graphs [Data: Reports (26)].

2. **Comprehensive and Diverse Responses**:
   By employing graph-based indexing and summarization techniques, GraphRAG ensures that the generated responses are both comprehensive and diverse, outperforming traditional summarization metho

In [10]:
# inspect the data used to build the context for the LLM responses
result.context_data["reports"]

,id,title,occurrence weight,content,rank
0,23,Natural Language Processing and Information Re...,0.444444,# Natural Language Processing and Information ...,9.0
1,73,Graph RAG and Community Summarization in NLP a...,0.333333,# Graph RAG and Community Summarization in NLP...,9.0
2,78,Few-Shot Examples and Their Role in Enhancing ...,0.166667,# Few-Shot Examples and Their Role in Enhancin...,9.0
3,35,Natural Language Processing and Information Re...,0.111111,# Natural Language Processing and Information ...,9.0
4,80,Natural Language Processing and Information Re...,0.111111,# Natural Language Processing and Information ...,9.0
...,...,...,...,...,...
78,82,Retrieval-Generation Synergy in Large Language...,0.055556,# Retrieval-Generation Synergy in Large Langua...,9.0
79,40,LLAMA and Its Impact on Natural Language Proce...,0.055556,# LLAMA and Its Impact on Natural Language Pro...,9.0
80,30,Natural Language Processing and Information Re...,0.055556,# Natural Language Processing and Information ...,9.0
81,101,Natural Language Processing and Information Re...,0.055556,# Natural Language Processing and Information ...,9.0


In [11]:
# inspect number of LLM calls and tokens
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")

LLM calls: 7. LLM tokens: 75225
